# Projet Data

## Sommaire

- <a href='#sectionIntro'>Introduction</a>
- <a href='#sectionVRP'>Le VRP</a>
- <a href='#sectionHeuristics'>Les heuristiques</a>
- <a href='#sectionSolution'>Solution</a>
    - <a href='#sectionInstanceGenration'>Génération d'instances</a>
    - <a href='#sectionAlgorithm'>Algorithme du recuit</a>
    - <a href='#sectionSolutionStats'>Etude de la solution</a>
- <a href='#sectionStats'>Statistiques</a>
- <a href='#sectionConclusion'>Conclusion</a>

<a id='sectionIntro'></a>
## Introduction

Ce Notebook a pour but d'expliquer notre démarche tout au long de ce projet Bug Data. Dans un premier temps nous discuterons du VRP, sa définition mathématique, comment il modélise notre problème et de ses différentes variantes. Ensuite nous étudierons les heuristiques, leur utilité et les différents types d'heuristiques. 


Introduction du projet et des attendus 


enjeux du VRP

<a id='sectionVRP'></a>
## Le VRP


Le VRP, ou Vehicule Routing Problem est un nom générique donné à une classe de problème de recherche opérationnelle et d'optimisation combinatoire. Le problème de base est formulé de la manière suivante : déterminer les différentes tournées d'une flotte de véhicules afin de livrer une liste de clients et de retourner à leur point de départ. Le but étant généralement de minimiser le coût des livraisons, mais on peut aussi choisir d'autres critères comme le temps où l'empreinte écologique.

![VRP](img/VRP.gif)


Ce problème est une extension du problème du voyageur de commerce (TSP) : il reprend le même principe mais ajoute plusieurs véhicules au lieu d'un seul pour le TSP. Cette contrainte fait place le VRP dans la catégorie NP-Hard.

Afin de résoudre le problème du VRP il faut le définir mathématiquement : 

![Démonstration mathématique](img/maths.png)

Aujourd'hui le VRP est un problème central dans les domaines des transports et de la logistique et il le sera encore pour les années futures. En effet optimiser ses livraisons est très important pour les entreprises des domaines cités précédemment car cela leur permet de réduire leurs coûts. De ce fait il existe de nombreuses variantes du VRP avec chacune des contraintes que l'on peut trouver dans des situations réelles :
- Capacited VRP (CVRP) : tous les véhicules ont une limite sur la quantité d'objets qu'ils peuvent transporter
- VRP with Time Window (VRPTM) : chaque client doit être livré dans une certaine plage horaire
- Multiple Depot VRP (MDVRP) : les véhicules peuvent partir de plusieurs dépôts
- VRP with Pick-Up and Delivering (VRPPD) : les clients peuvent renvoyer des éléments au passage des véhicules
- Split Delivery VRP (SDVRP) : un client peut être livré par plusieurs véhicules
- Stochastic VRP (SVRP) : on ajoute des valeurs aléatoires au problème (par exemple que les clients ont une probabilité p d'être présents)
- Periodic VRP (PVRP) : au lieu de faire les livraisons sur un seul jour, on peut livrer sur N jours

Pour ce projet nous avons décidés de choisir le problème du CVRP car il nous permet de remplir deux contraintes : utiliser k camions pour faire les livraisons et la prise en compte de la capacité des camions et de l'encombrement des objets.

<a id='sectionHeuristics'></a>
## Les heuristiques

Comme dit précédemment le problème du VRP est un problème NP-Hard, ce qui signifie que les ressources nécessaires pour le résoudre augmentent exponentiellement avec la taille de ses entrées. Pour résoudre ce problème dans un temps raisonnable nous allons donc utiliser des méthodes heuristiques. Les heuristiques sont des méthodes de calculs souvent utilisées pour résoudre des problèmes NP-Hard car elles permettent de trouver une solution assez proche de la solution optimale en un temps raisonnable. Il existe différents types d'heuristiques qui peuvent être comparées avec les critères suivants :
- Qualité du résultat
- Coût de l'heuristique (temps / mémoire)

Nous avons vu différents types d'heuristiques, chacune ayant des avantages et des inconvénients.

### Les algorithmes gloutons
Les algorithmes gloutons suivent le principe suivant : à chaque étape ils choisissent un optimum local. Ils ont en général un coût assez faible mais en général ils n'aboutissent pas à un optimal global, comme illustré dans l'exemple ci-dessous.

![Algorithme Glouton](img/greedy.png)
Ici on part du point A et on cherche à monter selon la plus forte pente. Avec ce type d'algorithme on atteint le point m qui est un maximum local mais pas global.


### La recherche Tabou
Le principe du Tabou est d'effectuer une recherche sur ses voisins, puis de prendre la valeur qui optimise la fonction objectif, tout en évitant les valeurs par lesquelles l'on est déjà passé. Cet algorithme permet de sortir d'un optimum local pour potentiellement trouver un minimum global. A chaque fois que l'on visite un nouveau point, on l'ajoute dans une file FIFO ce qui va permettre à l'algorithme de se souvenir des points qu'il a déjà traversé tout (dans la limite des n derniers points avec n la taille de la file).
C'est algorithme converge un peu plus lentement qu'un algorithme glouton mais il permet d'obtenir de meilleurs résultats.

### Le recuit simulé
Le recuit simulé se base sur la recherche tabou mais introduit une notion de température, plus la température est élevée, plus l'algorithme a de chance de prendre une solution de moins bonne qualité. L'algorithme se découpe en 3 phases :
- On choisit un point s au hasard sur la courbe à minimiser (pour avoir une valeur de départ) et on choisit un température T assez élevée.
- On fait une recherche sur les voisins de s. Si la nouvelle solution est meilleure alors on la garde, sinon on calcule la probabilité d'accepter une solution moins bonne à l'aide de la température T. Dans les deux on décrémente la température T.
- Quand la température a atteint un certain seuil décidé par l'utilisateur, le programme s'arrête.

L'exemple suivant montre la recherche d'un maximum sur une courbe avec la méthode du recuit simulé, avec en bleu l'optimum global à un instant t, en rouge la valeur de s, et en noir la valeur des voisins de s
![Recuit Simulé](img/recuit.gif)



### Les algorithmes génétiques
La dernière catégorie d'heuristiques que nous allons étudier sont les algorithmes génétiques. Ces derniers se basent sur le principe de la sélection naturelle afin de proposer une solution à un problème. Il existe une multitude de variantes d'algorithmes génétiques mais ils ont tous une base commune :
![Algorithme Génétique](img/genetic.jpg)

La première étape d'un algorithme génétique consiste à générer une population initiale d'individus (en se servant de valeurs aléatoires ou de valeurs par défaut). Ensuite les individus sont évalués par une fonction que l'on peut assimiler à une fonction objectif. Cette fonction va déterminer à quel point un individu est susceptible de se faire sélectionner pour servir de base à la prochaine population. Vient ensuite une étape de sélection qui va retenir des individus afin de les faire passer à la génération suivante (plus un individu a eu un bon score à sa fonction d'évaluation, plus il est susceptible d'être sélectionné). Ces deux étapes sont une reproduction du principe de sélection naturelle. Vient ensuite une étape de reproduction, dont le but va être de mélanger les caractéristiques des individus sélectionnés (échange de caractéristique, ajout / retrait de caractéristiques en fonction de valeurs aléatoires, ...) afin d'obtenir une nouvelle population. Cette étape peut être assimilé aux différentes mutations rencontrées dans le monde biologique.
Le programme s'arrête en fonction d'une condition qui peut être soit un nombre fixe d'itérations soit basée sur les caractéristiques de la population.

## Choix de l'algorithme
Pour ce projet notre choix s'est porté sur l'algorithme du recuit simulé. En effet il nous permet d'obtenir des solutions assez proches de la solution optimale dans un temps de calcul qui reste raisonnable pour notre projet. Les algorithmes gloutons et tabou donnant des solutions avec un écart très important par rapport à la solution optimale.

PARLER DE L4ALGO GENETIQUE

<a id='sectionSolution'></a>
## Solution

Comme dit précédemment notre solution s'appuie sur un algorithme de recuit simulé. Dans cette partie nous allons détaillées les différentes étapes de notre solution ainsi que tous les modules qui l'accompagne.

<a id='sectionInstanceGenration'></a>
### Génération d'instances
Afin de tester notre algorithme il nous a fallu un moyen de générer différents cas de tests pour notre programme. Nous avons donc créé un algorithme qui permet de générer des instances du problème du CVRP. La structure de donnée que nous avons utilisée est la suivante :

In [1]:
instance = {
    'trucksCount' : int,
    'trucksCapacity' : int,
    'nodes' : [int],
    'matrix' : [[int]]
}

Le script ci-dessous permet d'importer les différents modules et de les configurer. Il comporte aussi des fonctions utilitaires.

In [2]:
import logging
import random
import time
import csv

from math import hypot
from parse import parse

#matplotlib
import matplotlib.pyplot as plt
from matplotlib import pylab

#stats
import numpy as np
from numpy import arange,array,ones
from scipy import stats

RESULTS_FILE = "results.csv"

logging.basicConfig(filename='logs.log',
                        format='%(asctime)s %(levelname)-8s %(message)s',
                        level=logging.INFO,
                        datefmt='%Y-%m-%d %H:%M:%S')

def raiseIfNone(var):
    if var is None:
        raise Exception('Cannot be None')

### Algorithme de génération de données aléatoires

L'algorithme ci-dessous permet de générer des instances du problème CVRP avec des valeurs aléatoires. On peut modifier les bornes inférieures et supérieures des valeurs générées à l'aide de constantes au de variables. L'algorithme utilise un système de log afin de garder une trace de tous les jeux de données qui ont été généré. L'aléatoire est généré à partir d'une seed qui peut être spécifiée par l'utilisateur ou générée aléatoirement si l'utilisateur ne spécifie aucune valeur. C'est cette seed qui est stockée dans le fichier de log. Pendant la génération on fait aussi attention à ne pas générer des cas impossibles (par exemple en mettant un poids trop grand pour le nombre de camions et leur capacité).

In [3]:
MIN_NB_OF_TRUCKS = 2
MAX_NB_OF_TRUCKS = 5

MIN_TRUCK_CAPACITY = 50
MAX_TRUCK_CAPACITY = 100

MIN_NB_OF_NODES = 10
MAX_NB_OF_NODES = 10

MIN_NODE_Y = 0
MAX_NODE_Y = 100

MIN_NODE_X = 0
MAX_NODE_X = 100

MIN_DEMAND = 1
MAX_DEMAND = 25

def generateFromSeed(seed=None):
    if seed is None:
        random.seed()
        seed = random.random()

    random.seed(seed)
    logging.info('Seed : {0}'.format(seed))
    logging.info('Setup : {0}, {1}, {2}, {3}, {4}, {5}, {6}, {7}, {8}, {9}, {10}, {11}'.format(MIN_NB_OF_TRUCKS,
                                                                                              MAX_NB_OF_TRUCKS,
                                                                                              MIN_TRUCK_CAPACITY,
                                                                                              MAX_TRUCK_CAPACITY,
                                                                                              MIN_NB_OF_NODES,
                                                                                              MAX_NB_OF_NODES,
                                                                                              MIN_NODE_X,
                                                                                              MAX_NODE_X,
                                                                                              MIN_NODE_Y,
                                                                                              MAX_NODE_Y,
                                                                                              MIN_DEMAND,
                                                                                              MAX_DEMAND))
    
    citiesCount = random.randint(MIN_NB_OF_NODES, MAX_NB_OF_NODES)
        
    instance = {
        'trucksCount' : random.randint(MIN_NB_OF_TRUCKS, MAX_NB_OF_TRUCKS),
        'trucksCapacity' : random.randint(MIN_TRUCK_CAPACITY, MAX_TRUCK_CAPACITY),
        'nodes' : [],
        'matrix' : [[0]*citiesCount for i in range(citiesCount)]
    }

    totalDemand = 0
    totalDeliveryCapacity = instance['trucksCount'] * instance['trucksCapacity']

    for i in range(0, citiesCount):
        nodeX = random.randint(MIN_NODE_X, MAX_NODE_X)
        nodeY = random.randint(MIN_NODE_Y, MAX_NODE_Y)
        demand = random.randint(MIN_DEMAND, MAX_DEMAND)
        instance['nodes'].append({'id':i, 'x':nodeX, 'y':nodeY, 'demand':demand})

        totalDemand += demand
        if totalDemand > totalDeliveryCapacity:
            instance['trucksCapacity'] += math.ceil((totalDemand - totalDeliveryCapacity) / instance['trucksCount'])

    for fromNode in instance['nodes']:
        for toNode in instance['nodes'][fromNode['id']:citiesCount]:
            dist = int(hypot(toNode['x'] - fromNode['x'], toNode['y'] - fromNode['y']))
            instance['matrix'][fromNode['id']][toNode['id']] = dist
            instance['matrix'][toNode['id']][fromNode['id']] = dist
            
    return instance

### Algorithme de génération à partir d'un fichier
Afin de faire des statistiques sur la qualité de notre solution, nous devons utiliser notre algorithme sur des cas de test dont la solution optimale est connue, puis comparer cette solution avec celle de notre algorithme. Le programme suivant permet de créer une instance du problème à partir d'un fichier dont le nom est passé en paramètre.<br/><br/>
Ces fichiers ont le format suivant :<br/>
NAME : string - le nom du cas de test<br/>
COMMENT : (Augerat et al, No of trucks: int - nombre de camions, Optimal value: int - optimum global)<br/>
TYPE : CVRP - Non utilisé (tous nos cas de tests sont en CVRP)<br/>
DIMENSION : int - nombre de noeuds du graphe<br/>
EDGE_WEIGHT_TYPE : EUC_2D (non utilisé) <br/>
CAPACITY : int (capacité de chaque camion)<br/>
NODE_COORD_SECTION <br/>
 id x y (répété n fois en fonction du nombre de noeuds du graphe)<br/>
DEMAND_SECTION <br/>
id demand (répété n fois en fonction du nombre de noeuds du graphe<br/>
DEPOT_SECTION <br/>
 id (id du noeud qui représente le dépot)

In [4]:
def retrieveFromFile(fileName):
    instance = {
        'nodes' : list()
    }

    citiesCount = None

    with open(fileName, 'rt') as myFile:
        
        if myFile is None:
            raise Exception('Cannot open file \'{0}\''.format(fileName))
        
        logging.info('FileName : {0}'.format(fileName))
        
        line = myFile.readline()
        while line:
            if line.startswith("NAME : "):
                instance['name'] = parse('NAME : {}', line)[0]
            elif line.startswith("COMMENT : "):
                temp, instance['trucksCount'], instance['optimalValue'] = parse('COMMENT : {} No of trucks: {}, Optimal value: {})', line)            
            elif line.startswith("TYPE : "):
                pass
            elif line.startswith("DIMENSION : "):
                citiesCount = int(parse('DIMENSION : {}', line)[0])
            elif line.startswith("EDGE_WEIGHT_TYPE : "):
                pass
            elif line.startswith("CAPACITY : "):
                instance['trucksCapacity'] = int(parse('CAPACITY : {}', line)[0])
            elif line.startswith("NODE_COORD_SECTION "):
                raiseIfNone(citiesCount)
                for i in range(0, citiesCount):
                    line = myFile.readline()
                    id, x, y = parse(' {} {} {}', line)
                    instance['nodes'].append({'id':int(id) - 1, 'x':int(x), 'y':int(y)})
            
            elif line.startswith("DEMAND_SECTION "):
                raiseIfNone(citiesCount)
                for i in range(0, citiesCount):
                    line = myFile.readline()
                    id, demand = parse('{} {}', line)
                    instance['nodes'][int(id) - 1]['demand'] = demand[:-1]
                    
            elif line.startswith("DEPOT_SECTION "):
                line = myFile.readline()
                instance['depotNodeId'] = int(parse(' {}', line)[0]) - 1
                break
                
            line = myFile.readline()

    if citiesCount is None:
        raise Exception('CitiesCount cannot be None, verify that the input file as a valid format')

    instance['matrix'] = [[0]*citiesCount for i in range(citiesCount)]

    for fromNode in instance['nodes']:
        for toNode in instance['nodes'][fromNode['id']:citiesCount]:
            dist = int(hypot(toNode['x'] - fromNode['x'], toNode['y'] - fromNode['y']))
            instance['matrix'][fromNode['id']][toNode['id']] = dist
            instance['matrix'][toNode['id']][fromNode['id']] = dist
            
    return instance

<a id='sectionAlgorithm'></a>
### Partie Nico
Nico va faire la partie sur sa solution

Notre solution, comment elle fonctionne

In [5]:
def run():
    
    file = open(RESULTS_FILE, "w", newline='')
    try:
        writer = csv.writer(file)
        writer.writerow(('Time', 'Result', 'ExpectedResult', 'CitiesCount', 'TrucksCount', 'Iterations', 'Temperature', 'Coef'))
        
        #put this block in a for loop
        start_time = time.time()
        instance = generateFromSeed()
        #fonction de nico
        
        #logging results
        result = 800
        expectedResult = 500
        iterations = 0
        temperature = 0
        coef = 0
        writer.writerow((time.time() - start_time, result, expectedResult, len(instance['nodes']), instance['trucksCount'], iterations, temperature, coef))
    finally:
        file.close()
    
run()

<a id='sectionSolutionStats'></a>
### Performance de l'algorithme
Etude du niveau de complexité + stats

Prog stats + automatisation récolte des stats

Analyse de la solution niveau complexité

<a id='sectionStats'></a>
## Statistiques

Pour chaque instance de CVRP que notre algorithme resoud, il stocke le résultat dans un fichier CSV. Chaque entrée contient des informations qui permettent de faire des statistiques sur notre algorithme. On y retrouve le temps d'exécution, le résultat trouvé par notre algorithme et le résultat optimal (si le jeu de données est connu), et les paramètres du recuit (nombre d'itérations, tempérture de départ, ...).

In [6]:
def calculateLinearRegression(xvalues, yvalues, windowTitle):
    xvalues = np.array(xvalues)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(xvalues, yvalues)
    line = slope * xvalues + intercept

    print('Y(x) = {0}x + {1}'.format(slope, intercept))

    plt.plot(xvalues, yvalues, 'o', xvalues, line)
    pylab.title(windowTitle)

def retrieveStatsFromFile():
    
    data = [[]]
    
    file = open(RESULTS_FILE, "rt")
    try:
        reader = csv.reader(file)
        next(reader)
        
        data = list(reader)
        
    finally:
        file.close()
        
    return data

<a id='sectionConclusion'></a>
## Conclusion